In [ ]:
from pyspark.sql.functions import col 
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
configs = {
    "fs.azure.account.auth.type": "OAuth", 
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider", 
    "fs.azure.account.oauth2.client.id": "PASTE CLIENT-ID ", 
    "fs.azure.account.oauth2.client.secret": 'PASTE SECERT-ID', 
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/PASTE TANENT-ID/oauth2/token"
} 


In [ ]:
dbutils.fs.mount( 
source = "abfss://my-data-container@qadeersa .dfs.core.windows.net",   # contrainer@storageAccountName 
mount_point = "/mnt/azuredatafolder",
    extra_configs = configs)

In [ ]:
%fs ls "/mnt/azuredatafolder"

In [ ]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/azuredatafolder/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/azuredatafolder/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/azuredatafolder/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/azuredatafolder/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/azuredatafolder/raw-data/teams.csv")

In [ ]:
athletes.show()

In [ ]:
athletes.printSchema()


In [ ]:
entriesgender.printSchema()


In [ ]:
# entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
#     .withColumn("Male",col("Male").cast(IntegerType()))\
#     .withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()
     

In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

In [ ]:
# Write the data
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/azuredatafolder/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/azuredatafolder/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/azuredatafolder/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/azuredatafolder/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/azuredatafolder/transformed-data/teams")
     